In [1]:

import pandas as pd
import numpy as np
import altair as alt
from tqdm import tqdm
from general_consts import *
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [32]:
def get_df(path, table_name):
    df = pd.read_csv(fr"{path}/{table_name}.csv")
    df[CPUColumns.TIME] = pd.to_datetime(df[CPUColumns.TIME], unit='s')
    df[CPUColumns.TIME] += pd.Timedelta(hours=2)
    return df

def load_tables(path):
    cpu_df = get_df(path, TableNames.CPU)
    memory_df = get_df(path, TableNames.MEMORY)
    all_processes_df = get_df(path, TableNames.ALL_PROCESSES)
    disk_df = get_df(path, TableNames.DISK)
    splunk_logs_df = pd.read_csv(fr"{path}/output.csv")
    return cpu_df, memory_df, all_processes_df, disk_df, splunk_logs_df
    
def draw_plot(df, y, path, plot_name):
    alt.renderers.enable('altair_saver', fmts=['png'])
    chart = alt.Chart(df).mark_line(interpolate='linear').encode(
        alt.Y(f"{y}:Q", scale=alt.Scale(domain=[min(df[y]), max(df[y])])),
        alt.X(f"{CPUColumns.TIME}:T"),
    ).properties(
        width=2100,
        height=300
    )
    # Create PNG image data and then write to a file
    with alt.data_transformers.enable('default'):
        chart.save(f"{path}/graphs/{plot_name}.png", engine="altair_saver")
    return chart

In [37]:
for measurment_no in range(1, 7):
    path = fr"/home/green-sec/Repositories/GreenSecurity-FirstExperiment/Dell Inc. Linux 5.15.0-67-generic/Splunk Enterprise SIEM/Power Saver Plan/One Scan/Measurement {measurment_no}"
    cpu_df, memory_df, all_processes_df, disk_df, splunk_logs_df = load_tables(path)
    cpu = draw_plot(cpu_df, CPUColumns.USED_PERCENT, path, 'cpu')
    disc_r_c = draw_plot(disk_df, DiskIOColumns.READ_COUNT, path, 'disk_read_count')
    disc_w_c = draw_plot(disk_df, DiskIOColumns.WRITE_COUNT, path, 'disk_write_count')
    disc_r_b = draw_plot(disk_df, DiskIOColumns.READ_BYTES, path, 'disk_read_bytes')
    disc_w_b = draw_plot(disk_df, DiskIOColumns.WRITE_BYTES, path, 'disk_write_bytes')
    splunk_logs_df['_time'] = pd.to_datetime(splunk_logs_df['_time'])
    splunk_logs_df['_time'] = splunk_logs_df['_time'].dt.tz_localize(None)

    # Define number of bins and opacity range
    n_bins = 200
    logs = alt.Chart(splunk_logs_df).mark_bar(opacity=0.3, color='green').encode(
        alt.Y("count():Q"),
        alt.X("_time:T", bin=alt.Bin(maxbins=n_bins)),
    ).properties(
        width=2100,
        height=300,
    )
    # Create PNG image data and then write to a file
    with alt.data_transformers.enable('default'):
        logs.save(f"{path}/graphs/logs_dist.png", engine="altair_saver")
        alt.layer(cpu,logs).resolve_scale(
        y = 'independent').save(f"{path}/graphs/cpu_logs_merge.png", engine="altair_saver")
        
        alt.layer(disc_r_c,logs).resolve_scale(
        y = 'independent').save(f"{path}/graphs/disc_r_c_logs_merge.png", engine="altair_saver")
        
        alt.layer(disc_w_c,logs).resolve_scale(
        y = 'independent').save(f"{path}/graphs/disc_w_c_logs_merge.png", engine="altair_saver")
        
        alt.layer(disc_r_b,logs).resolve_scale(
        y = 'independent').save(f"{path}/graphs/disc_r_b_logs_merge.png", engine="altair_saver")
        
        alt.layer(disc_w_b,logs).resolve_scale(
        y = 'independent').save(f"{path}/graphs/disc_w_b_logs_merge.png", engine="altair_saver")
        

In [33]:
# splunk_logs_df['_time'] = pd.to_datetime(splunk_logs_df['_time']).dt.strftime('%H:%M:%S')
# Define number of bins and opacity range
n_bins = 200
opacity_range = [0.2, 0.8]


chart = alt.Chart(splunk_logs_df).mark_bar(opacity=0.3, color='green').encode(
    alt.Y("count():Q"),
    alt.X("_time:T", bin=alt.Bin(maxbins=n_bins))
).properties(
    width=2100,
    height=300,
    # title='Bar Chart with Adjustable Opacity'
)
with alt.data_transformers.enable('default'):
    alt.layer(draw_plot(cpu_df, CPUColumns.USED_PERCENT, path, 'cpu'),chart).resolve_scale(
        y = 'independent'
    ).save(f"{path}/graphs/merge.png", engine="altair_saver")
